In [1]:
!python --version

Python 3.10.14


In [2]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: '--quiet'


In [3]:
!pip install --upgrade --quiet  yandexcloud

In [4]:
import os
from dotenv import load_dotenv
from langchain_community.embeddings.yandex import YandexGPTEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import LLMChain
from langchain_community.llms import YandexGPT

In [5]:
with open('.env', 'w') as f:
    f.write('YANDEX_API_KEY=A...Write your key\n')
    f.write("YANDEX_FOLDER_ID=b...Write your ID\n") 
    
    

In [6]:
load_dotenv('.env')
YANDEX_API_KEY = os.getenv('YANDEX_API_KEY')
YANDEX_FOLDER_ID = os.getenv('YANDEX_FOLDER_ID')



In [7]:
llm = YandexGPT(api_key=YANDEX_API_KEY, folder_id=YANDEX_FOLDER_ID)

In [8]:
embedding = YandexGPTEmbeddings(api_key=YANDEX_API_KEY, folder_id=YANDEX_FOLDER_ID)

In [9]:
pdf_directory = "data/"
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

text_splitter = CharacterTextSplitter(
    separator=".",
    chunk_size=2500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

all_pages = []

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_directory, pdf_file)
    loader = PyPDFLoader(pdf_path)
    pages = loader.load_and_split(text_splitter)
    all_pages.extend(pages)

In [10]:
db = Chroma.from_documents(all_pages, embedding)
db

retriever = db.as_retriever(search_kwargs={"k": 7})

In [11]:
template = """
Вы - помощник с искусственным интеллектом.
Отвечайте, исходя из предоставленного контекста.

context: {context}
input: {input}
answer:
"""

In [12]:
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [15]:
query = "Каким должен быть угол отбортовки?"
response = retrieval_chain.invoke({"input": query})

In [16]:
print(response["answer"])

90°.
